# Εβδομάδα 4 - Ανάκτηση και προετοιμασία δεδομένων



Στο συγκεκριμένο notebook θα εξετάσουμε με ποιον τρόπο μπορούμε να ανακτήσουμε δεδομένα που βρίσκονται αναρτημένα στο διαδίκτυο (scraping), σε μορφή HTML. Στην συνέχεια θα δοκιμάσουμε να περιηγηθούμε μέσω υπερσυνδέσεων για να δημιουργήσουμε ένα δείγμα από online περιεχόμενο, με την τεχνική "spidering" ή με την χρήση APIs, (Application Programming Interfaces), που παρέχονται ορισμένες φορές από τις διάφορες εταιρείες σε προγραμματιστές και παρέχουν πρόσβαση στο περιεχόμενό τους. 

Στην συνέχεια των μαθημάτων θα δοκιμάσουμε regular expressions, για να "καθαρίσουμε" τα κείμενα από ανεπιθύμητο περιεχόμενο όπως σημεία στίξης, μορφοποιήσεις κ.λπ.

Στο τέλος θα εξερευνήσουμε τους τρόπους με τους οποίους μπορούμε να αποθηκέυσουμε τα δεδομένα, ώστε να είναι έτοιμα για ανάλυση. 

Για το συγκεκριμένο notebook θα χρειαστούμε τα εξής πακέτα: 

In [ ]:
#Τα παρακάτω πακέτα θα εγκατασταθούν με την εντολή pip στον υπολογιστή (στον σέρβερ υπάρχουν ήδη)

import requests #βιβλιοθήκη για http requests
import bs4 #ονομάζεται `beautifulsoup4`, και είναι ένας html parser
import pandas #δημιουργεί DataFrames
import docx #διαβάζει MS doc files, για εγκατάσταση θέλουμε το `python-docx`

#Tα παρακάτω υπάρχουν στην Python
import re #regexs
import urllib.parse #ένωση urls
import io #κάνει τα http requests να μοιάζουν με αρχεία
import json #API responses
import os.path #Τσεκάρει ότι υπάρχουν τα αρχεία 
import os #Δημιουργεί directories

Θα δουλέψουμε επίσης με τα παρακάτω αρχεία και urls

In [ ]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Πριν ξεκινήσει η ανάλυση περιεχομένου πρέπει να έχουμε το περιεχόμενο στην κατοχή μας. Μερικές φορές μπορεί να φτάσει στα χέρια μας ήδη "καθαρό" και έτοιμο για χρήση ως αρχείο κειμένου και άλλες φορές θα χρειαστεί να το κατεβάσουμε από το διαδίκτυο.

Για αρχή ας δοκιμάσουμε να κατεβάσουμε μια σελίδα από το wikipedia για ανάλυση περιεχομένου. 

Η σελίδα βρίσκεται στο (https://en.wikipedia.org/wiki/Content_analysis)

Θα χρησιμοποιήσουμε μία HTTP GET request για το συγκεκριμένο url, μια GET request είναι ένα απλό αίτημα στον server για να μας δώσει το περιεχόμενο αυτού του url. Ένα άλλο είδος είναι το POST request και πρόκειται για ένα αίτημα στο οποίο ζητάμε εμείς από τον server να πάρει ένα αρχείο που του δίνουμε. 

Παρόλο που η Python έχει ήδη μια βιβλιοθήκη για GET requests, εμείς θα χρησιμοποιούμε την 
[_requests_](http://docs.python-requests.org/en/master/) γιατί είναι πιο εύχρηστη.

In [ ]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

`'Response [200]'` σημαίνει ότι ο server ανταποκρίθηκε σε αυτό που του ζητήσαμε. Αν παίρναμε άλλον αριθμό (π.χ. 404) θα σήμαινε ότι υπάρχει κάποιο error. 

Υπάρχει λίστα με όλους τους διαθέσιμους κωδικούς από HTTP response codes εδώ: 
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). 

To response διαθέτει όλα τα data που μας έστειλε ο server, το περιεχόμενο του website και το HTTP header. Εμάς μας ενδιαφέρει το περιεχόμενο που μπορούμε να το προσπελάσουμε με την εντολή `.text`.

In [ ]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

Αυτό δεν είναι το αποτέλεσμα που περιμέναμε, καθώς πρόκειται για την αρχή της  HTML του website. Η γλώσσα αυτή διαβάζεται από υπολογιστές, για το λόγο αυτό θα χρησιμοποιήσουμε έναν parser για να τη διαβάσουμε. Για το parsing θα χρησιμοποιήσουμε το [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/).

In [ ]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])

Αυτό είναι καλύτερο από το προηγούμενο αποτέλεσμα, αλλά και πάλι περιέχει πολλά κενά και όχι μόνο κείμενο. Αυτό συνέβη γιατί ζητήσαμε όλη την webpage, και όχι μόνο το κείμενο.

Εμείς θέλουμε μόνο το κείμενο και για να πετύχουμε να πάρουμε μόνο αυτό, πρέπει να ελέγξουμε την html. Ένας εύκολος τρόπος να το κάνουμε αυτό είναι να επισκεφθούμε το website με έναν browser και να χρησιμοποιήσουμε το inspection ή το "view source tool". Σε περίπτωση που υπάρχει javascript ή κάποιο άλλο είδος δυναμικού loading στην σελίδα, τότε πρέπει να ελέγξουμε αυτό που λαμβάνει η Python και χρειαζόμαστε τη βιβλιοθήκη `requests`.

In [ ]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Τώρα ας ανοίξουμε το αρχείο (`wikipedia_content_analysis.html`) που μόλις δημιουργήσαμε με τον web browser. Πρέπει να μοιάζει όπως το original αλλά χωρίς τις εικόνες και τη μορφοποίηση. 

Επειδή δεν υπάρχει τίποτα θέσφατο στη δημιουργία μιας ιστοσελίδας, το να καταλάβουμε τι πραγματικά είναι χρήσιμο για μας είναι Τέχνη. Κοιτώντας σε αυτή την σελίδα φαίνεται πως το κυρίως κείμενο είναι μέσα στα `<p>`(paragraph) tags που βρίσκονται μέσα στο `<body>` tag.

In [ ]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Τώρα έχουμε όλο το κείμενο χωρισμένο σε παραγράφους.  

Μας μένει να κάνουμε κάτι ακόμα πριν αρχίσουμε την επεξεργασία, να βγάλουμε τα references indicators (`[2]`, `[3]` , κ.λπ). Για να γίνει αυτό χρειάζεται λίγο regular expression (regex).

In [ ]:
contentParagraphs = []
for pTag in contentPTags:
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#μετατροπή σε DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

Τώρα έχουμε ένα `DataFrame` που περιέχει όλα τα σχετικά κείμενα από την ιστοσελίδα. 

Επειδή δεν είστε ακόμα experts σε regex, πρόκειται για έναν τρόπο να πραγματοποιούμε συγκεκριμένες αναζητήσεις μέσα στο κείμενο. 

Μια μηχανή regex αντιλαμβάνεται το search pattern, στην παραπάνω περίπτωση το `'\[\d+\]'` και κάποιο string από τα paragraph texts που έχουμε ορίσει. Στην συνέχεια τσεκάρει ένα ένα τα γράμματα από το  input string για να ελέγξει αν ταιριάζουν στην αναζήτηση. Εδώ το regex `'\d'` ταιρίαζει αριθμούς ( (digits), ενώ το `'\['` και το  `'\]'` "πιάνει" τις αγκύλες σε κάθε πλευρά.

In [ ]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

Στην Python το regex package (`re`) συνήθως επιστρέφει `Match` objects (μπορεί να έχουμε διάφορα pattern hits σε ένα μόνο `Match`), για να πάρουμε το string που ταίριαξε στο pattern που ζητήσαμε μπορούμε να χρησιμοποιήσουμε τη μέθοδο `.group()`. Επειδή θέλουμε το πρώτο θα ζητήσουμε το  0'. Στην πληροφορική ΠΑΝΤΑ το πρώτο είναι το 0. 

In [ ]:
print(regexResults.group(0))

Πήραμε τον πρώτο αριθμό, αν θέλαμε όλο το block αριθμών θα προσθέταμε ένα `'+'` , το οποίο ζητά ένα ή παραπάνω εμφανίσεις του   προηγούμενου αριθμού. 

In [ ]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'όχι αρθμός, όχι αρθμός, , αριθμοί 2134567890, όχι αρθμός ')
print(regexResults.group(0))

Τώρα έχουμε όλο το block των αριθμών. 

Αν θέλετε να μάθετε περισσότερα για τα Python  regex δείτε [re docs](https://docs.python.org/3/library/re.html) και το μικρό 
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

Το καλύτερο tutorial βρίσκεται εδώ: https://regexone.com


# Αράχνες (Spidering)

Τι θα κάναμε αν θέλαμε διαφορετικές σελίδες από την wikipedia. Θα χρειαζόμασταν όλες τις διαφορετικές url για την καθεμιά από αυτές. Αυτό που μπορούμε να κάνουμε είναι να βρούμε σελίδες που συνδέονται με άλλες σελίδες μέσω κάποιου link. Ας προσπαθήσουμε να βρούμε τι  links υπάρχουν στην σελίδα που αναφέρεται στο content analysis.

Για να προχωρήσουμε πρέπει πρώτα να αναζητήσουμε όλα τα `<a>` (anchor) tags με `href`
(hyperlink references) που υπάρχουν μέσα στα `<p>` tags. Το `href` μπορεί να έχει διάφορες 
(http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-it-used) μορφές (https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML), οπότε πρέπει να είστε προσεκτικοί.

Σε γενικές γραμμές θέλουμε να εξάγουμε ολόκληρα (absolute) ή σχετικά (relative) links. Ένα absolute link είναι αυτό που το ακολουθούμε απευθείας χωρίς μετατροπή, ενώ το relative link πρέπει πρώτα να ενωθεί με την βάση (base url) με append. Το Wikipedia χρησιμοποιεί relative urls για τα εσωτερικά links: 

In [ ]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#Θέλουμε επίσης να ξέρουμε από που προήλθαν τα links, άρα θέλουμε:
#αριθμό παραγράφου
#τη λέξη στην οποία βρίσκεται τo link
for paragraphNum, pTag in enumerate(contentPTags):
    # μόνο τα hrefs που οδηγούν σε wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #Πρέπει να εξάγουμε το url από το <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #το wikipedia_base_url είναι η βάση στην οποία θα ενώσουμε με το urllib τα links
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

Θα προσθέσουμε αυτά τα 2 νέα στοιχεία στο DataFrame `contentParagraphsDF`, άρα πρέπει να προσθέσουμε  2 ακόμη στήλες για paragraph numbers και sources.

In [ ]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

Μετά προσθέτουμε 2 νέες στήλες στο `Dataframe` και ορίζουμε ένα function για να φέρει κάθε linked page και να προσθέσει το κείμενο της στο DataFrame μας.

In [ ]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Φτιάχνουμε ένα dict στο οποίο θα σώσουμε τα data πριν τα αποθηκεύσουμε στο DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Τώρα παίρνουμε την σελίδα 
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #Με το enumerating παίρνουμε τον αριθμό της παραγράφου
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #το ίδιο regex με πριν
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

Και το τρέχουμε στη λίστα με τα link tags

In [ ]:
for urlTuple in otherPAgeURLS[:3]:
    #Το ignore_index σημαίνει ότι οι δείκτες δεν θα διαγράφονται μετά από κάθε append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

## API (Tumblr)

Οι ιδιοκτήτες των websites δεν θέλουν να κάνετε scraping στα sites τους. Αν δεν είστε προσεκτικοί και 'χτυπάτε' συνεχόμενα το site, αυτό θα μοιάζει με DOS attack. Ορισμένα sites θέλουν αυτοματοποιημένα  εργαλεία για να έχουμε πρόσβαση στα δεδομένα τους και οι ίδιοι δημιουργούν [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). Ένα API
συγκκριμενοποιεί τη διαδικασία με την οποία παρέχει τις πληροφορίες. Συνήθως γίνεται μέσω μιας [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service.

Ένα καλό παράδειγμα για εμάς είναι το [Tumblr](https://www.tumblr.com), που μας παρέχουν 
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) το οποίο μας επιτρέπει να διαβάζουμε posts χωρίς να χρειάζεται να κάνουμε δύσκολο html parsing.

Μπορούμε εύκολα να πάρουμε τα πρώτα 20 posts από ένα blog μέσω μιας http GET request στο
`'http://{blog}.tumblr.com/api/read/json'`, το `{blog}` είναι το όνομα του blog που στοχεύουμε. Ας πάρουμε μερικές δημοσιεύσεις από εδώ [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Το όνομα του Tumblr blog βρίσκεται στο URL 'lolcats-lol-cat').

In [ ]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

Μπορεί να μην φαίνεται εύκολο, αλλά είναι πολύ ευκολότερο από την html! Αυτό που βλέπουμε είναι ένα
[JSON](https://en.wikipedia.org/wiki/JSON) ένα 'human readable' κείμενο βασισμένο στα δεδομένα που στάλθηκαν σε ένα transmission format βασισμένο στην γλώσσα javascript. Ευτυχώς για εμάς, μπορούμε εύκολα να το μετατρέψουμε σε ένα python `dictionary`.

In [ ]:
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

Αν διαβάσουμε το [API specification](https://www.tumblr.com/docs/en/api/v1), θα δούμε ότι μπορούμε να πάρουμε πολλά πράγματα με ένα GET request. Πρώτα θα πάρουμε κάποια posts με το id number. Ας ξεκινήσουμε με το post
`146020177084`.

In [ ]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Αν δεν μπορείτε να το δείτε κάντε refresh. Τώρα μπορούμε να πάρουμε το κείμενο από όλα τα posts μαζί με τα σχετικά metadata, όπως το post date, το caption και τα tags. Επίσης μπορούμε να πάρουμε τα 
links των φωτογραφιών.

In [ ]:
#Βάζουμε έναν αριθμό max σε περίπτωση που το blog έχει εκατομμύρια εικόνες.
#Tο max θα είναι πολλαπλάσιο του 50
def tumblrImageScrape(blogName, maxImages = 200):
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

   
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #Όλες τις πληροφορίες θα τις αποθηκεύσουμε στο τέλος σε ένα DataFrame.
    #Στο Tumblr documentation μπορείτε να βρείτε πώς θα πάρετε και άλλες πληροφροίες.
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Το Tumblr μας περιορίζει σε max 50 posts σε κάθε request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #Παίρνουμε uncleaned data, δεν μπορούμε να τα εμπιστευτούμε.
            #Δηλαδή, όλα τα posts δεν είναι απαραίτητο ότι περιέχουν όλες τις πληροφορίες που ζητάμε να πάρουμε.
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Διαβάστε τα λάθη
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)